In [1]:
######################################     Modules     #######################################
# MODULES
import sys
from time import process_time
import pickle
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact

sys.path.append('../.')
import verbose_module
sys.path.append('../../../.')
import pIRPgym

path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/pIRPgym/'
experiments_path = '/Users/juanbeta/My Drive/Research/Supply Chain Analytics/Experiments/Flower Agent/'

sizes = [5,10,15,20]

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11


# Supplier Availability

In [2]:
def plot_distributions(size):
    with open(experiments_path+f'M{size}.pkl', 'rb') as file:
        data = pickle.load(file)
    inst_gen = data[2]
    pIRPgym.Visualizations.RoutingV.plot_supplier_distributions(inst_gen.q_parameters)

interact(plot_distributions,size=widgets.Dropdown(options=sizes,value=5,description='Inst. size:'))

interactive(children=(Dropdown(description='Inst. size:', options=(5, 10, 15, 20), value=5), Output()), _dom_c…

<function __main__.plot_distributions(size)>

In [ ]:
# 

In [5]:
def plot_locations(size):
    with open(experiments_path+f'M{size}.pkl', 'rb') as file:
        data = pickle.load(file)
    inst_gen = data[2]
    pIRPgym.Visualizations.RoutingV.render_routes(inst_gen,[])

interact(plot_locations,size=widgets.Dropdown(options=sizes,value=5,description='Inst. size:'))

interactive(children=(Dropdown(description='Inst. size:', options=(5, 10, 15, 20), value=5), Output()), _dom_c…

<function __main__.plot_locations(size)>